In [1]:
!pip install -q -U git+https://github.com/fabienfrfr/tptt@dev
!pip install -q -U lighteval

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 k

In [2]:
# only in kaggle for HF
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

In [3]:
base_model_name="meta-llama/Llama-3.2-1B"
tokenizer_name=base_model_name

# saving
username = "ffurfaro"
model_name = '/Titans-' + base_model_name.split('/')[1]
repo_id = username + '/' + model_name.lstrip('/')

In [4]:
import torch, tptt
from transformers import AutoModelForCausalLM, AutoTokenizer
from tptt.modeling_tptt import TpttModel

2025-06-05 19:22:57.364587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749151377.622119      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749151377.694239      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
model_tptt = TpttModel.from_pretrained(repo_id, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(repo_id, token=hf_token)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [6]:
device = 0 if torch.cuda.is_available() else -1
model_tptt.to(f"cuda:{device}" if device != -1 else "cpu")
model_tptt.eval()

pipe = tptt.TpttPipeline(model=model_tptt.backbone, tokenizer=tokenizer, device=None)

result = pipe("Bonjour, I'm Fabien Furfaro, ", max_new_tokens=512, do_sample=True, eos_token_id=None)
print(result[0]["generated_text"],'\n', len(result[0]["generated_text"]))

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bonjour, I'm Fabien Furfaro, 23 years old and living in Paris. I speak French, English and Spanish, and I love to travel, read, watch movies and listen to music. I'm looking for a partner to share my adventures with, and I'm always open to new 
 243


In [12]:
eva_tptt_code = """
import os
import torch
from lighteval.models.abstract_model import LightevalModel
from transformers import AutoTokenizer
from tptt.modeling_tptt import TpttModel

class ModelInfo:
    def __init__(self, model_name, model_sha=None, model_dtype=None, model_size=None):
        self.model_name = model_name
        self.model_sha = model_sha or ""
        self.model_dtype = model_dtype or "float32"
        self.model_size = model_size or "unknown"

class EvaTpttModel(LightevalModel):
    def __init__(self, config):
        super().__init__()
        repo_id = os.environ.get("REPO_ID")
        hf_token = os.environ.get("HF_TOKEN")
        device = os.environ.get("DEVICE", "cpu")
        self.model = TpttModel.from_pretrained(repo_id, token=hf_token).to(device)
        self._tokenizer = AutoTokenizer.from_pretrained(repo_id, token=hf_token)
        self.device = device
        self.model.eval()
        try:
            n_params = sum(p.numel() for p in self.model.parameters())
        except Exception:
            n_params = "unknown"
        self.model_info = ModelInfo(
            model_name="EvaTpttModel",
            model_sha="custom",
            model_dtype=str(self.model.dtype) if hasattr(self.model, "dtype") else "float32",
            model_size=n_params
        )

    @property
    def tokenizer(self):
        return self._tokenizer

    @property
    def max_length(self):
        return getattr(self._tokenizer, "model_max_length", 2048)

    def add_special_tokens(self, tokens_dict):
        return self._tokenizer.add_special_tokens(tokens_dict)

    def greedy_until(self, requests, max_tokens=None, stop_sequences=None):
        results = []
        for req in requests:
            prompt = getattr(req, "prompt", getattr(req, "context", ""))
            inputs = self._tokenizer(prompt, return_tensors="pt").to(self.device)
            with torch.no_grad():
                output_ids = self.model.generate(
                    **inputs,
                    max_new_tokens=max_tokens or 32,
                    eos_token_id=self._tokenizer.eos_token_id,
                    pad_token_id=self._tokenizer.pad_token_id,
                )
            text = self._tokenizer.decode(output_ids[0], skip_special_tokens=True)
            if stop_sequences:
                for stop in stop_sequences:
                    idx = text.find(stop)
                    if idx != -1:
                        text = text[:idx]
                        break
            results.append((text, None))
        return results

    def loglikelihood(self, requests, log=True):
        results = []
        for req in requests:
            if hasattr(req, "context") and hasattr(req, "continuation"):
                full_text = req.context + (req.continuation or "")
            elif hasattr(req, "prompt"):
                full_text = req.prompt
            else:
                raise ValueError(f"Requête inattendue : {req}")
            inputs = self._tokenizer(full_text, return_tensors="pt").to(self.device)
            labels = inputs["input_ids"].clone()
            with torch.no_grad():
                outputs = self.model(**inputs, labels=labels)
                log_likelihood = -outputs.loss.item() * labels.size(1)
            results.append((log_likelihood, True))
        return results


    def loglikelihood_rolling(self, requests):
        results = []
        for req in requests:
            text = getattr(req, "text", getattr(req, "prompt", ""))
            tokens = self._tokenizer.encode(text)
            stride = 512
            ll = 0.0
            for i in range(0, len(tokens), stride):
                input_ids = torch.tensor([tokens[i:i+stride]]).to(self.device)
                with torch.no_grad():
                    outputs = self.model(input_ids=input_ids, labels=input_ids)
                    ll += -outputs.loss.item() * input_ids.size(1)
            results.append((ll, True))
        return results

    def loglikelihood_single_token(self, requests):
        results = []
        for req in requests:
            context = getattr(req, "context", "")
            token = getattr(req, "token", "")
            context_ids = self._tokenizer.encode(context, return_tensors="pt").to(self.device)
            token_id = self._tokenizer.encode(token, add_special_tokens=False)[0]
            input_ids = torch.cat([context_ids, torch.tensor([[token_id]]).to(self.device)], dim=1)
            labels = input_ids.clone()
            with torch.no_grad():
                outputs = self.model(input_ids=input_ids, labels=labels)
                ll = -outputs.loss.item()
            results.append((ll, True))
        return results
"""

with open("eva_tptt_model.py", "w") as f:
    f.write(eva_tptt_code)

In [ ]:
import os
import torch
from lighteval.pipeline import Pipeline, PipelineParameters, ParallelismManager
from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.custom.custom_model import CustomModelConfig

os.environ["REPO_ID"] = repo_id
os.environ["HF_TOKEN"] = hf_token
os.environ["DEVICE"] = "cuda" if torch.cuda.is_available() else "cpu"
CUSTOM_MODEL_PATH = "eva_tptt_model.py"


evaluation_tracker = EvaluationTracker(
    output_dir="./results",
    save_details=True
)

pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.ACCELERATE,
    max_samples=5
)

model_config = CustomModelConfig(
    model_name="eva-tptt-model",
    model_definition_file_path=CUSTOM_MODEL_PATH,
)

tasks="leaderboard|truthfulqa:mc|0|0"

pipeline = Pipeline(
    tasks=tasks,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model_config=model_config
)

pipeline.evaluate()
pipeline.save_and_push_results()
pipeline.show_results()